<a href="https://colab.research.google.com/github/laraimad/Machine-learning-assignment-/blob/main/prediction_for_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor



In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path='/content/drive/MyDrive/mldata0.xlsx'
df = pd.read_excel(file_path, sheet_name=2)
df

/usr/local/lib/python3.11/dist-packages/openpyxl/reader/workbook.py:102: UserWarning: Defined names for sheet index 19 cannot be located
  warn(f"Defined names for sheet index {idx} cannot be located")
/usr/local/lib/python3.11/dist-packages/openpyxl/reader/workbook.py:102: UserWarning: Defined names for sheet index 12 cannot be located
  warn(f"Defined names for sheet index {idx} cannot be located")


,Unnamed: 0,2018 Cases,2019 Cases,2020 Cases,2021 Cases,2022 Cases
0,Whooping cough,9,257,95,43,17
1,Tetanus,10,0,0,7,6
2,Measles,1156,1024,34,327,247
3,Mumps,106,187,188,150,101
4,Rubella,53,36,27,86,85
5,Chickenpox,4389,3663,1633,969,1457
6,Brucellosis,5455,4257,2372,2400,2543
7,Amoebic dysentery,3351,2592,1594,1685,964
8,Bacillary dysentery (Shigellosis),71,76,36,55,25
9,Salmonella,2044,3187,1451,2323,1443


In [ ]:
X = df.iloc[:, 1:-1].values  # Features: 2018-2021 cases
y = df.iloc[:, -1].values  # Target: 2022 cases


# Splitting the data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
models= {
    'Support Vector Machine': SVR(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'MLP': MLPRegressor()
}


kf = KFold(n_splits=4, shuffle=False)
OriginalModelPerformance = {} # Initialize the dictionary to store results
for name, mdl in models.items():
    scores = cross_val_score(mdl, X, y, cv=kf, n_jobs=-1)  # 5-fold cross-validation
    print(f"{name}:  {scores.mean():.3%}, {scores.std():.3%}")

Support Vector Machine:  -2640.088%, 4240.523%
Random Forest:  54.904%, 38.214%
Gradient Boosting:  64.079%, 14.192%
MLP:  9.805%, 75.649%


In [ ]:
scaler = StandardScaler()
Xs = scaler.fit_transform(X)  # Only for models that need scaling

In [ ]:
models= {

    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'MLP': MLPRegressor()
}


kf = KFold(n_splits=4, shuffle=True, random_state=42)
OriginalModelPerformance = {} # Initialize the dictionary to store results
for name, mdl in models.items():
    scores = cross_val_score(mdl, Xs, y, cv=kf, n_jobs=-1)
    print(f"{name}:  {scores.mean():.3%}, {scores.std():.3%}")

Random Forest:  71.574%, 24.341%
Gradient Boosting:  74.610%, 11.435%
MLP:  -108.746%, 57.109%


In [ ]:
pca = PCA(n_components=0.95)
XSR = pca.fit_transform(Xs)

In [ ]:
models= {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(),
    'MLP': MLPRegressor()
}


kf = KFold(n_splits=4, shuffle=True, random_state=42)
OriginalModelPerformance = {} # Initialize the dictionary to store results
for name, mdl in models.items():
    scores = cross_val_score(mdl, XSR, y, cv=kf, n_jobs=-1)
    print(f"{name}:  {scores.mean():.3%}, {scores.std():.3%}")

Random Forest:  83.973%, 15.503%
Gradient Boosting:  65.575%, 11.122%
MLP:  -110.791%, 57.525%


In [ ]:
# Predict 2023 Cases using the trained Random Forest model
X_future = X  # Use all features (2018-2021) for prediction
X_future_pca = pca.transform(X_future)  # Apply PCA to match model input
predicted_2023 = rf_pca.predict(X_future_pca)  # Predict using the model
print(predicted_2023)

[  47.54   30.63  191.44   76.54   67.04 1545.68 2679.22 1545.68   41.84
  929.28 3187.06]


In [ ]:



df_predictions = pd.DataFrame({
    "Disease": df.iloc[:, 0],   # Disease names
    "Predicted 2023 Cases": predicted_2023.astype(int)  # Convert to integer values
})

# Display results
print(df_predictions.to_string(index=False))  # Print without index

                          Disease  Predicted 2023 Cases
                   Whooping cough                    47
                          Tetanus                    30
                          Measles                   191
                            Mumps                    76
                          Rubella                    67
                       Chickenpox                  1545
                      Brucellosis                  2679
                Amoebic dysentery                  1545
Bacillary dysentery (Shigellosis)                    41
                       Salmonella                   929
                     Dengue fever                  3187
